<a href="https://colab.research.google.com/github/AShipulin/usurt/blob/main/%D0%9B%D0%9F%D0%A2%D0%A1_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BB%D0%B0%D0%BD_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
from ortools.linear_solver import pywraplp

# Создаем решатель
solver = pywraplp.Solver.CreateSolver('SCIP')

# Данные задачи
suppliers = ['a1', 'a2']
supply = {'a1': 10, 'a2': 20}

consumers = ['b1', 'b2', 'b3']
demand = {'b1': 8, 'b2': 13, 'b3': 9}

costs = {
    ('a1', 'b1'): 7,
    ('a1', 'b2'): 9,
    ('a1', 'b3'): 21,
    ('a2', 'b1'): 20,
    ('a2', 'b2'): 15,
    ('a2', 'b3'): 16
}

# Создаем переменные
x = {}
for s in suppliers:
    for c in consumers:
        x[(s, c)] = solver.IntVar(0, solver.infinity(), f'x_{s}_{c}')

# Ограничения по запасам
for s in suppliers:
    solver.Add(
        sum(x[(s, c)] for c in consumers) <= supply[s],
        f'Supply_{s}'
    )

# Ограничения по спросу
for c in consumers:
    solver.Add(
        sum(x[(s, c)] for s in suppliers) >= demand[c],
        f'Demand_{c}'
    )

# Целевая функция: минимизация стоимости
solver.Minimize(
    sum(x[(s, c)] * costs[(s, c)] for s in suppliers for c in consumers)
)

# Решаем задачу
status = solver.Solve()

# Вывод результатов
if status == pywraplp.Solver.OPTIMAL:
    print('Оптимальное решение найдено!')
    print(f'Общая стоимость: {solver.Objective().Value()}\n')

    print('План перевозок:')
    for s in suppliers:
        for c in consumers:
            if x[(s, c)].solution_value() > 0:
                print(f'Из {s} в {c}: {x[(s, c)].solution_value()} ед.')
else:
    print('Решение не найдено.')

Оптимальное решение найдено!
Общая стоимость: 383.0

План перевозок:
Из a1 в b1: 8.0 ед.
Из a1 в b2: 2.0 ед.
Из a2 в b2: 11.0 ед.
Из a2 в b3: 9.0 ед.
